In [2]:
import os
import json
import pandas as pd

In [3]:
def konsolidiere_seen_daten(ordner_pfad):
    """
    Liest JSON-Dateien aus einem Ordner und erstellt einen konsolidierten DataFrame.

    Args:
        ordner_pfad (str): Pfad zum Ordner mit den JSON-Dateien.

    Returns:
        pandas.DataFrame: Konsolidierter DataFrame mit den Daten aus allen JSON-Dateien.
    """
    data = []
    for filename in os.listdir(ordner_pfad):
        if filename.endswith(".json"):
            file_path = os.path.join(ordner_pfad, filename)
            with open(file_path, 'r') as f:
                try:
                    json_data = json.load(f)
                    # Extrahiere relevante Daten
                    yearly_data = json_data['yearly']
                    for location, rcp_data in yearly_data.items():
                        for rcp, values in rcp_data.items():
                            years = values['x']
                            y_ave = values['y_ave']
                            y_min = values['y_min']
                            y_max = values['y_max']

                            # Erstelle eine Liste von Dictionaries für jedes Jahr
                            for i in range(len(years)):
                                data.append({
                                    'See': filename.replace(".json", ""),  # Name der Datei als Seenname
                                    'Location': location,
                                    'RCP': rcp,
                                    'Jahr': years[i],
                                    'Durchschnitt': y_ave[i],
                                    'Minimum': y_min[i],
                                    'Maximum': y_max[i]
                                })
                except json.JSONDecodeError as e:
                    print(f"Fehler beim Lesen der JSON-Datei {filename}: {e}")
                except KeyError as e:
                    print(f"Fehler beim Zugriff auf Schlüssel in {filename}: {e}")

    # Erstelle den DataFrame
    df = pd.DataFrame(data)
    return df

# Beispielaufruf
ordner_pfad = "../data/see_data" 
konsolidierter_df = konsolidiere_seen_daten(ordner_pfad)
print(konsolidierter_df)

# Optional: Speichere den DataFrame als CSV-Datei/
konsolidierter_df.to_csv("../data/see_daten.csv", index=False)


Empty DataFrame
Columns: []
Index: []


## See- Koordinaten korrigiert

In [6]:
df = pd.read_csv("../data/see_lat_lon_final.csv")
print(df[df["See"].str.contains("Zurich", case=False)])


             See  Latitude  Longitude
10  Lower-Zurich   47.2500     8.6833
27  Upper-Zurich   47.3667     8.6167


In [7]:
df.loc[df["See"] == "Upper-Zurich", "Latitude"] = 47.2667
df.loc[df["See"] == "Upper-Zurich", "Longitude"] = 8.6000

df.loc[df["See"] == "Lower-Zurich", "Latitude"] = 47.2667
df.loc[df["See"] == "Lower-Zurich", "Longitude"] = 8.6000